# Gráficos de Cleveland: CO Congruentes vs Incongruentes por Categoría Ideológica

Este notebook genera **gráficos de Cleveland** para visualizar el Cambio de Opinión (CO) comparando respuestas **congruentes** e **incongruentes** con la ideología, agrupadas por **categoría ideológica**.

## Concepto:

### Congruencia Ideológica:
- **CO_Congruente**: Ítems Progresistas → Izquierda + Ítems Conservadores → Derecha
- **CO_Incongruente**: Ítems Progresistas → Derecha + Ítems Conservadores → Izquierda

### Visualización:
- **Punto azul (●)**: CO Congruente (ideológicamente consistente)
- **Punto rojo (●)**: CO Incongruente (ideológicamente inconsistente)
- **Línea conectando**: Muestra la diferencia
- **Color de línea**:
  - Verde: Mayor CO en Incongruentes (>0.1)
  - Gris: Similar (±0.1)
  - Naranja: Mayor CO en Congruentes (<-0.1)

## Interpretación:

- **Valores positivos**: Mayor cambio de opinión
- **Hipótesis**: Se espera mayor CO en ítems incongruentes (conflicto ideológico genera más cambios)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib.lines import Line2D
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✓ Librerías cargadas exitosamente')

## 1. Cargar Datos

In [ ]:
# Rutas a los archivos
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Procesados')
Archivo_Generales = os.path.join(Ruta_Base, 'Generales_con_Congruencia.xlsx')
Archivo_Ballotage = os.path.join(Ruta_Base, 'Ballotage_con_Congruencia.xlsx')

# Cargar datos
df_Generales = pd.read_excel(Archivo_Generales)
df_Ballotage = pd.read_excel(Archivo_Ballotage)

dfs = {
    'Generales': df_Generales,
    'Ballotage': df_Ballotage
}

print(f'✓ Datos cargados:')
print(f'  - Generales: {len(df_Generales)} registros')
print(f'  - Ballotage: {len(df_Ballotage)} registros')

# Verificar variables
vars_necesarias = ['CO_Congruente', 'CO_Incongruente', 'Categoria_PASO_2023']
for nombre, df in dfs.items():
    faltantes = [v for v in vars_necesarias if v not in df.columns]
    if faltantes:
        print(f'  ⚠️  {nombre}: Faltan variables {faltantes}')
    else:
        print(f'  ✅ {nombre}: Todas las variables presentes')

## 2. Preparar Datos para Gráfico

In [ ]:
# Definir categorías ideológicas
Categorias = [
    'Left_Wing',
    'Progressivism',
    'Centre',
    'Moderate_Right_A',
    'Moderate_Right_B',
    'Right_Wing_Libertarian'
]

Etiquetas_Categorias = {
    'Left_Wing': 'Left Wing',
    'Progressivism': 'Progressivism',
    'Centre': 'Centre',
    'Moderate_Right_A': 'Moderate Right A',
    'Moderate_Right_B': 'Moderate Right B',
    'Right_Wing_Libertarian': 'Right Wing Libertarian'
}

print(f'Categorías ideológicas: {len(Categorias)}')
for cat in Categorias:
    print(f'  - {Etiquetas_Categorias[cat]}')

In [ ]:
# Calcular promedios por categoría para cada dataset
def Preparar_Datos_Por_Categoria(df, categorias):
    """
    Calcula promedios de CO_Congruente e Incongruente por categoría.
    """
    datos = []
    
    for categoria in categorias:
        df_cat = df[df['Categoria_PASO_2023'] == categoria]
        
        if len(df_cat) > 0:
            media_cong = df_cat['CO_Congruente'].mean()
            media_incong = df_cat['CO_Incongruente'].mean()
            diferencia = media_incong - media_cong
            
            datos.append({
                'Categoria': categoria,
                'Etiqueta': Etiquetas_Categorias[categoria],
                'n': len(df_cat),
                'CO_Congruente': media_cong,
                'CO_Incongruente': media_incong,
                'Diferencia': diferencia
            })
    
    return pd.DataFrame(datos)

# Preparar datos para ambos datasets
datos_graficos = {}
for nombre, df in dfs.items():
    datos_graficos[nombre] = Preparar_Datos_Por_Categoria(df, Categorias)
    print(f'\n{nombre}:')
    print(datos_graficos[nombre][['Etiqueta', 'n', 'CO_Congruente', 'CO_Incongruente', 'Diferencia']])

## 3. Función para Crear Gráfico de Cleveland

In [ ]:
def Crear_Grafico_Cleveland_CO_Categorias(df, titulo, nombre_archivo=None, carpeta_destino='Graficos_Cleveland'):
    """
    Crea un gráfico de Cleveland mostrando CO Congruente vs Incongruente por categoría.
    """
    
    # Crear carpeta si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    
    # Ordenar por diferencia (descendente)
    df_sorted = df.sort_values('Diferencia', ascending=True).reset_index(drop=True)
    
    # Crear figura
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Parámetros visuales
    y_positions = np.arange(len(df_sorted))
    
    # Dibujar líneas conectando los puntos
    for idx, row in df_sorted.iterrows():
        cong_val = row['CO_Congruente']
        incong_val = row['CO_Incongruente']
        diferencia = row['Diferencia']
        
        # Determinar color de línea según diferencia
        if diferencia > 0.1:  # Más CO en Incongruentes
            color_linea = '#2ecc71'  # Verde
            alpha = 0.7
        elif diferencia < -0.1:  # Más CO en Congruentes
            color_linea = '#e74c3c'  # Rojo/Naranja
            alpha = 0.7
        else:  # Diferencia pequeña
            color_linea = '#95a5a6'  # Gris
            alpha = 0.4
        
        # Dibujar línea
        ax.plot([cong_val, incong_val], [idx, idx], 
                color=color_linea, linewidth=2, alpha=alpha, zorder=1)
    
    # Dibujar puntos de Congruente
    ax.scatter(df_sorted['CO_Congruente'], y_positions, 
               s=150, c='#3498db', marker='o', 
               edgecolors='white', linewidths=2,
               label='Congruente', zorder=3, alpha=0.9)
    
    # Dibujar puntos de Incongruente
    ax.scatter(df_sorted['CO_Incongruente'], y_positions, 
               s=150, c='#e74c3c', marker='o', 
               edgecolors='white', linewidths=2,
               label='Incongruente', zorder=3, alpha=0.9)
    
    # Línea vertical en x=0
    ax.axvline(x=0, color='black', linestyle='--', linewidth=0.8, alpha=0.3, zorder=0)
    
    # Configurar ejes
    ax.set_yticks(y_positions)
    ax.set_yticklabels([row['Etiqueta'] for _, row in df_sorted.iterrows()], fontsize=11)
    
    ax.set_xlabel('Cambio de Opinión (promedio)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Categoría Ideológica', fontsize=12, fontweight='bold')
    ax.set_title(titulo, fontsize=14, fontweight='bold', pad=20)
    
    # Grid
    ax.grid(True, axis='x', alpha=0.3, linestyle=':', linewidth=0.5)
    ax.set_axisbelow(True)
    
    # Leyenda personalizada
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#3498db', 
               markersize=11, label='Congruente', markeredgecolor='white', markeredgewidth=1.5),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#e74c3c', 
               markersize=11, label='Incongruente', markeredgecolor='white', markeredgewidth=1.5),
        Line2D([0], [0], color='#2ecc71', linewidth=2.5, label='Mayor Incong (>0.1)'),
        Line2D([0], [0], color='#95a5a6', linewidth=2.5, label='Similar (±0.1)'),
        Line2D([0], [0], color='#e74c3c', linewidth=2.5, label='Mayor Cong (<-0.1)')
    ]
    
    ax.legend(handles=legend_elements, loc='lower right', fontsize=10, 
              framealpha=0.95, edgecolor='gray')
    
    # Ajustar layout
    plt.tight_layout()
    
    # Guardar
    if nombre_archivo:
        ruta_completa = os.path.join(carpeta_destino, f'{nombre_archivo}.png')
        plt.savefig(ruta_completa, dpi=300, bbox_inches='tight', facecolor='white')
        print(f'✅ Gráfico guardado: {ruta_completa}')
    
    plt.show()
    
    return fig, ax

## 4. Gráficos por Dataset

In [ ]:
# Crear gráfico para Generales
print('Generando gráfico para Generales...\n')
fig_gen, ax_gen = Crear_Grafico_Cleveland_CO_Categorias(
    datos_graficos['Generales'],
    titulo='CO Congruente vs Incongruente - Generales',
    nombre_archivo='Cleveland_CO_Congruente_vs_Incongruente_Generales'
)

In [ ]:
# Crear gráfico para Ballotage
print('Generando gráfico para Ballotage...\n')
fig_bal, ax_bal = Crear_Grafico_Cleveland_CO_Categorias(
    datos_graficos['Ballotage'],
    titulo='CO Congruente vs Incongruente - Ballotage',
    nombre_archivo='Cleveland_CO_Congruente_vs_Incongruente_Ballotage'
)

## 5. Análisis Estadístico

In [ ]:
print('='*70)
print('ANÁLISIS ESTADÍSTICO: CO CONGRUENTE VS INCONGRUENTE')
print('='*70)

for nombre, df_datos in datos_graficos.items():
    print(f'\n📊 {nombre}:')
    print('-'*70)
    
    print(f'\nEstadísticas Generales:')
    print(f'  Promedio CO_Congruente: {df_datos["CO_Congruente"].mean():.4f}')
    print(f'  Promedio CO_Incongruente: {df_datos["CO_Incongruente"].mean():.4f}')
    print(f'  Diferencia promedio: {df_datos["Diferencia"].mean():.4f}')
    
    # Distribución
    mayor_incong = len(df_datos[df_datos['Diferencia'] > 0.1])
    similares = len(df_datos[(df_datos['Diferencia'] >= -0.1) & (df_datos['Diferencia'] <= 0.1)])
    mayor_cong = len(df_datos[df_datos['Diferencia'] < -0.1])
    
    print(f'\nDistribución:')
    print(f'  Mayor en Incongruentes: {mayor_incong} categorías')
    print(f'  Similar: {similares} categorías')
    print(f'  Mayor en Congruentes: {mayor_cong} categorías')
    
    # Test t pareado
    t_stat, p_value = stats.ttest_rel(df_datos['CO_Incongruente'], df_datos['CO_Congruente'])
    
    print(f'\nTest t Pareado:')
    print(f'  Estadístico t: {t_stat:.4f}')
    print(f'  Valor p: {p_value:.4f}')
    
    if p_value < 0.05:
        print(f'  ✅ SIGNIFICATIVO (p < 0.05)')
        if df_datos['Diferencia'].mean() > 0:
            print(f'     Mayor CO en Incongruentes')
        else:
            print(f'     Mayor CO en Congruentes')
    else:
        print(f'  ❌ NO SIGNIFICATIVO (p ≥ 0.05)')
    
    print('\n' + '-'*70)

print('\n' + '='*70)

## 6. Categorías con Mayor Diferencia

In [ ]:
print('='*70)
print('CATEGORÍAS CON MAYOR DIFERENCIA')
print('='*70)

for nombre, df_datos in datos_graficos.items():
    print(f'\n📊 {nombre}:')
    print('-'*70)
    
    print(f'\nMayor diferencia (Incong > Cong):')
    top_incong = df_datos.nlargest(3, 'Diferencia')[['Etiqueta', 'CO_Congruente', 'CO_Incongruente', 'Diferencia']]
    print(top_incong.to_string(index=False))
    
    print(f'\nMayor diferencia (Cong > Incong):')
    top_cong = df_datos.nsmallest(3, 'Diferencia')[['Etiqueta', 'CO_Congruente', 'CO_Incongruente', 'Diferencia']]
    print(top_cong.to_string(index=False))
    
    print('\n' + '-'*70)

print('\n' + '='*70)

## 7. Guardar Tablas Resumen

In [ ]:
# Crear carpeta de salida
Carpeta_Salida = os.path.join(os.getcwd(), '..', 'Data', 'Resultados_Cleveland')
if not os.path.exists(Carpeta_Salida):
    os.makedirs(Carpeta_Salida)

# Guardar tablas
for nombre, df_datos in datos_graficos.items():
    archivo = os.path.join(Carpeta_Salida, f'Resumen_CO_Congruente_vs_Incongruente_{nombre}.xlsx')
    df_datos.to_excel(archivo, index=False)
    print(f'✅ Guardado: {archivo}')

## 8. Resumen Final

In [ ]:
print('='*70)
print('RESUMEN: CO CONGRUENTE VS INCONGRUENTE POR CATEGORÍA')
print('='*70)

print('\n📊 Análisis completado:')
print(f'  - Datasets analizados: 2 (Generales y Ballotage)')
print(f'  - Categorías por dataset: {len(Categorias)}')
print(f'  - Gráficos generados: 2 (Cleveland plots)')

print('\n📁 Archivos generados:')
print('  - Cleveland_CO_Congruente_vs_Incongruente_Generales.png')
print('  - Cleveland_CO_Congruente_vs_Incongruente_Ballotage.png')
print('  - Resumen_CO_Congruente_vs_Incongruente_Generales.xlsx')
print('  - Resumen_CO_Congruente_vs_Incongruente_Ballotage.xlsx')

print('\n💡 Interpretación:')
print('  - CO_Congruente: Cambios ideológicamente consistentes')
print('  - CO_Incongruente: Cambios ideológicamente inconsistentes')
print('  - Hipótesis: Mayor CO en incongruentes (conflicto ideológico)')

print('\n' + '='*70)
print('✅ ANÁLISIS COMPLETADO')
print('='*70)